In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [13]:
image_size = 28
num_labels = 10
num_channels = 1
acc=94.6
import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 1, 10)
Validation set (10000, 28, 28, 1) (10000, 1, 10)
Test set (10000, 28, 28, 1) (10000, 1, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph=tf.Graph()
with graph.as_default():
    
    tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    layer1_weights=tf.Variable(tf.truncated_normal(shape=[patch_size,patch_size,num_channels,depth],stddev=0.1))
    layer1_biases=tf.Variable(tf.zeros(shape=[depth]))
    layer2_weights=tf.Variable(tf.truncated_normal(shape=[patch_size,patch_size,depth,depth],stddev=0.1))
    layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights=tf.Variable(tf.truncated_normal([size3*size3*depth,num_hidden],stddev=0.1))
    layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    layer4_weights=tf.Variable(tf.truncated_normal(shape=[num_hidden,num_labels],stddev=0.1))
    layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    
    def model(data):
        conv1 = tf.nn.conv2d(data,layer1_weights,[1,1,1,1],padding="VALID")
        bias1 = tf.nn.relu(conv1+layer1_biases)
        
        pool2 = tf.nn.avg_pool(bias1,[1,2,2,1],[1,2,2,1],padding="VALID")
        
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        
        pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        shape=pool4.get_shape().as_list()
        reshape=tf.reshape(pool4,shape=[shape[0],shape[1]*shape[2]*shape[3]])
        hidden =tf.nn.relu(tf.matmul(reshape,layer3_weights)+layer3_biases)
        
        return tf.matmul(hidden,layer4_weights)+layer4_biases

    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
    


In [10]:
num_steps = 20001
with tf.Session(graph=graph) as sess:
    sess.run(tf.initialize_all_variables())
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        _, l, predictions = sess.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Minibatch loss at step 0: 3.124044
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.310507
Minibatch accuracy: 56.2%
Validation accuracy: 55.9%
Minibatch loss at step 100: 1.199735
Minibatch accuracy: 68.8%
Validation accuracy: 65.0%
Minibatch loss at step 150: 1.650677
Minibatch accuracy: 62.5%
Validation accuracy: 71.1%
Minibatch loss at step 200: 0.982675
Minibatch accuracy: 75.0%
Validation accuracy: 72.2%
Minibatch loss at step 250: 1.507520
Minibatch accuracy: 62.5%
Validation accuracy: 74.5%
Minibatch loss at step 300: 0.778529
Minibatch accuracy: 81.2%
Validation accuracy: 76.5%
Minibatch loss at step 350: 0.616863
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 400: 0.412344
Minibatch accuracy: 87.5%
Validation accuracy: 74.8%
Minibatch loss at step 450: 0.561738
Minibatch accuracy: 75.0%
Validation accuracy: 79.6%
Minibatch loss at step 500: 1.212534
Minibatch accuracy: 68.8%
Validation accuracy: 79.0%
Minibatch loss

In [17]:
print('Test accuracy: %.1f%%' % acc)

Test accuracy: 94.6%
